In [ ]:
# 해당 파일은 python 3.10.9에서 작성 되었습니다.

## 1. 중학 실력 Test 데이터

ISHERPA_2010 > TBL_WWW_SelfTest_Info, TBL_WWW_SelfTest_Wrong_Note_EduBase 테이블 사용

### 1-1. 데이터 불러오기

In [ ]:
import pandas as pd

dataset1 = pd.read_csv('./datasets/중학실력Test_문항별정오답.csv')
dataset1.head()

In [ ]:
dataset1['Test_Level'].unique()

In [ ]:
dataset1['Test_Name'].unique()[:20]

### 1-2. 데이터 컬럼 설명

### 1-3. 데이터 활용 아이디어

- 문항별 정오답을 예측해보기 (로지스틱 회귀로 분류)


- 시험 점수를 예측해보기 (선형 회귀)


## 2. 월간 학습자 통계 데이터

### 2-1. 데이터 불러오기

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset2 = pd.read_csv('./datasets/월간학습자통계.csv', index_col = 0)
dataset2.head()

In [ ]:
check_df = dataset2

print('rows x columns :', check_df.shape) # (행 개수, 열 개수)
# 칼럼별 unique 개수 확인
columns_info = dict()
columns = check_df.columns
for col in columns:
    columns_info[col] = len(check_df[col].unique())
data = [{'index': col, 'unique_cut': count} for col, count in columns_info.items()]
unique_cut_df = pd.DataFrame(data)
non_null_cut_df = pd.DataFrame(check_df.count()).rename(columns = {0 : 'non_null_cut'}).reset_index()# 칼럼별 비결측치 개수
info_df = pd.DataFrame(check_df.dtypes).rename(columns = {0 : 'dtype'}).reset_index() # 데이터 프레임 info
# 칼럼별 결측지 비율 확인
null_cnt_df = pd.DataFrame(check_df.isnull().sum()).rename(columns = {0 : 'null_cut'}).reset_index()
null_cnt_df['null_ratio'] = round(null_cnt_df['null_cut'] / len(check_df) * 100, 2)
# 합치기
unique_non_null_cut_df = pd.merge(unique_cut_df, non_null_cut_df)
unique_non_null_info_cut_df = pd.merge(unique_non_null_cut_df, info_df)
unique_non_null_info_null_cut_df = pd.merge(unique_non_null_info_cut_df, null_cnt_df).rename(columns = {'index' : 'columns'})
# describe - 통계
int_float_cnt  = unique_non_null_info_null_cut_df[(unique_non_null_info_null_cut_df['dtype'] == 'int64') |
                 (unique_non_null_info_null_cut_df['dtype'] == 'float64')]
pd.set_option('display.max_columns', len(int_float_cnt))
# 보여주기
display(unique_non_null_info_null_cut_df)
display(check_df.describe())

In [ ]:
dataset2['memberstatus_codename'].value_counts()

In [ ]:
# dataset2['total_caliper_learning_time'] 학습시간 합계 (캘리퍼)
# dataset2['test_average_score'] 학생별 평가 평균 점수 예측 (선형 회귀)

In [ ]:
# 캘리퍼 학습시간과 평가 평균 점수 결측치 제거
dataset2_1 = dataset2[['total_caliper_learning_time','test_average_score']].dropna()
dataset2_1

In [ ]:
# 캘리퍼 학습시간 60분 미만 제거
dataset2_1 = dataset2_1[dataset2_1['total_caliper_learning_time'] >= 60]
dataset2_1

In [ ]:
# # 캘리퍼 학습시간 60분 미만이면서 평가 평균 점수 10점 미만 제거
dataset2_1 = dataset2_1[(dataset2_1['total_caliper_learning_time'] >= 60) &
                        (dataset2_1['test_average_score'] >= 10)]
dataset2_1

In [ ]:
check_df = dataset2_1

print('rows x columns :', check_df.shape) # (행 개수, 열 개수)
# 칼럼별 unique 개수 확인
columns_info = dict()
columns = check_df.columns
for col in columns:
    columns_info[col] = len(check_df[col].unique())
data = [{'index': col, 'unique_cut': count} for col, count in columns_info.items()]
unique_cut_df = pd.DataFrame(data)
non_null_cut_df = pd.DataFrame(check_df.count()).rename(columns = {0 : 'non_null_cut'}).reset_index()# 칼럼별 비결측치 개수
info_df = pd.DataFrame(check_df.dtypes).rename(columns = {0 : 'dtype'}).reset_index() # 데이터 프레임 info
# 칼럼별 결측지 비율 확인
null_cnt_df = pd.DataFrame(check_df.isnull().sum()).rename(columns = {0 : 'null_cut'}).reset_index()
null_cnt_df['null_ratio'] = round(null_cnt_df['null_cut'] / len(check_df) * 100, 2)
# 합치기
unique_non_null_cut_df = pd.merge(unique_cut_df, non_null_cut_df)
unique_non_null_info_cut_df = pd.merge(unique_non_null_cut_df, info_df)
unique_non_null_info_null_cut_df = pd.merge(unique_non_null_info_cut_df, null_cnt_df).rename(columns = {'index' : 'columns'})
# describe - 통계
int_float_cnt  = unique_non_null_info_null_cut_df[(unique_non_null_info_null_cut_df['dtype'] == 'int64') |
                 (unique_non_null_info_null_cut_df['dtype'] == 'float64')]
# 보여주기
if len(unique_non_null_info_null_cut_df.columns) >= len(int_float_cnt.columns):
    len_number = len(unique_non_null_info_null_cut_df.columns)
else:
    len_number = len(int_float_cnt.columns)
pd.set_option('display.max_columns', len_number)            
display(unique_non_null_info_null_cut_df)
display(check_df.describe())

In [ ]:
# 캘리퍼 학습시간과 평가 평균 점수
X = dataset2_1["test_average_score"]
y = dataset2_1["total_caliper_learning_time"]
plt.plot(X, y, 'o')
plt.show()

In [ ]:
lin_reg = LinearRegression()
X = X.values.reshape(-1, 1)
lin_reg.fit(X, y)


In [ ]:
# 절편과 기울기
lin_reg.intercept_, lin_reg.coef_

In [ ]:
z = int(input('받고 싶은 점수(0 ~ 100)를 입력하세요: '))
predicted_score = lin_reg.predict([[z]])[0]

if z > 100 :
    print('100 이하의 점수를 입력하세요')
elif z < 0 :
    print('0점이 최저입니다.')
else :
    print(z, '\b점을 받으려면 아래의 시간(분)만큼 공부하시면 됩니다.')
    print(round(predicted_score, 2), '\b분')

In [ ]:
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 타이틀 한글 출력
plt.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# 유사한 학습 횟수를 가지는 학생 군집화
# media_count
# test_count
# wrong_count

In [ ]:
input_data = dataset2[['media_count', 'test_count', 'wrong_count']].dropna()

In [ ]:
gmm = GaussianMixture(n_components = 3, random_state = 42)
gmm_labels = gmm.fit_predict(input_data)

input_data['gmm_cluster'] = gmm_labels

In [ ]:
def ploting_score(df):
    f, ax = plt.subplots(3, 1)
    f.set_size_inches((10, 10))

    plt.subplots_adjust(wspace=0.3, hspace=0.3)

    ############ 1 ###############

    sns.histplot(df[df['gmm_cluster'] == 0]['media_count'].tolist(),
                color="tomato", ax=ax[0])
    sns.histplot(df[df['gmm_cluster'] == 1]['media_count'].tolist(),
                color="mediumorchid", ax=ax[0])
    sns.histplot(df[df['gmm_cluster'] == 2]['media_count'].tolist(),
                color="gold", ax=ax[0])
    ax[0].set_title('media_count')
    ax[0].set_xlim([-10, 110])

    ############ 2 ###############

    sns.histplot(df[df['gmm_cluster'] == 0]['test_count'].tolist(),
                color="tomato", ax=ax[1])
    sns.histplot(df[df['gmm_cluster'] == 1]['test_count'].tolist(),
                color="mediumorchid", ax=ax[1])
    sns.histplot(df[df['gmm_cluster'] == 2]['test_count'].tolist(),
                color="gold", ax=ax[1])
    ax[1].set_title('test_count')
    ax[1].set_xlim([-10, 110])

    ############ 3 ###############
    sns.histplot(df[df['gmm_cluster'] == 0]['wrong_count'].tolist(),
                color="tomato", ax=ax[2])
    sns.histplot(df[df['gmm_cluster'] == 1]['wrong_count'].tolist(),
                color="mediumorchid", ax=ax[2])
    sns.histplot(df[df['gmm_cluster'] == 2]['wrong_count'].tolist(),
                color="gold", ax=ax[2])
    ax[2].set_title('wrong_count')
    ax[2].set_xlim([-10, 110])

    plt.xlim(0, 100)
    plt.show()

ploting_score(input_data)

In [ ]:
input_data1 = input_data

In [ ]:
input_data = input_data.drop(columns = 'gmm_cluster')

In [ ]:
k = 5
kmeans = KMeans(n_clusters = k, init = 'k-means++',
                max_iter = 300, random_state = 0)

In [ ]:
X = dataset2["study_count"]
y = dataset2["test_average_score"]
plt.plot(X, y, 'o')
plt.show()

In [ ]:
import numpy as np
input_data_array = input_data.to_numpy()
input_data_2d = input_data_array.reshape(-1, 1)
kmeans.fit(input_data_2d)


In [ ]:
kmeans.fit(input_data)

In [ ]:
y_pred = kmeans.fit_predict(input_data)
y_pred

In [ ]:
input_data['cluster'] = y_pred
input_data

In [ ]:
kmeans.cluster_centers_

In [ ]:
def plot_clusters(X):
    plt.scatter(X[:, 0], X[:, 1], s = 1)
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$", rotation = 0)

plt.figure(figsize = (8, 4))
plot_clusters(input_data.values)  # Assuming input_data1 is a DataFrame
plt.gca().set_axisbelow(True)
plt.grid()

# save_fig("blobs_plot")
plt.show()

In [ ]:
def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize = 2)

def plot_centroids(centroids, weights = None, circle_color = 'w', cross_color = 'k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker = 'o', s = 35, linewidths = 8,
                color = circle_color, zorder = 10, alpha = 0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker = 'x', s = 2, linewidths = 12,
                color = cross_color, zorder = 11, alpha = 1)

def plot_decision_boundaries(clusterer, X, resolution = 1000, show_centroids = True,
                             show_xlabels = True, show_ylabels = True):
    # 바탕 그리기
    mins = X.min(axis = 0) - 0.1
    maxs = X.max(axis = 0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent = (mins[0], maxs[0], mins[1], maxs[1]),
                cmap = "Pastel2")

    # 결정 경계 그리기
    plt.contour(Z, extent = (mins[0], maxs[0], mins[1], maxs[1]),
                linewidths = 1, colors = 'k')

    # 훈련 데이터셋과 센트로이드 그리기
    plot_data(X)
    if show_centroids:
        plot_centroids(clusterer.cluster_centers_)

    if show_xlabels:
        plt.xlabel("$x_1$")
    else:
        plt.tick_params(labelbottom = False)
    if show_ylabels:
        plt.ylabel("$x_2$", rotation = 0)
    else:
        plt.tick_params(labelleft = False)

plt.figure(figsize = (8, 4))
plot_decision_boundaries(kmeans, input_data.values)

plt.show()

In [ ]:
# # K-means로도 시각화를 해보고 싶었지만 방법을 찾다가 시간이 부족해서 못했습니다.ㅠㅠ

### 2-3. 데이터 활용 아이디어

- 학생별 평가 평균 점수 예측하기 (선형 회귀)


- 유사한 학습 횟수를 가지는 학생 군집화

## 3. 학생별 과목별 점수 데이터

4일차 GMM을 통한 학생 수준별 군집화 실습을 통해 사용했던 데이터! --> 중1 데이터


본 데이터는 중2 데이터

### 3-1. 데이터 불러오기

In [ ]:
dataset3 = pd.read_csv('./datasets/중2_AI진단평가_과목별점수.csv', index_col = 0)
dataset3.head()

### 3-2. 데이터 활용 아이디어

- 데이터 핸들링 하기 (시각화, groupby 등 다양한 전처리 연습하기)


- 유사 사용자 클러스터링 하기